# Chapter 6: Object References, Mutability, and Recycling

Variables are labels, not boxes. This chapter covers object identity, value, and aliasing

In [1]:
# variables are labels
a = [1, 2, 3] # a is binded to the list
b = a # b binds to a which is binded to the list

a.append(4) # change whatever a binds to (the list)
b # since b binds to a, any changes to a are reflected in b

[1, 2, 3, 4]

In [3]:
# aliasing

seuss = {
    'name': 'Dr. Seuss',
    'born': 1904,
}

theodore = seuss

theodore is seuss, id(theodore), id(seuss)

(True, 4428689984, 4428689984)

In [4]:
theodore['balance'] = 100
seuss

{'name': 'Dr. Seuss', 'born': 1904, 'balance': 100}

In [ ]:
# whos this guy??
impostor = {'name': 'Dr. Seuss', 'born': 1904, 'balance': 100}

# they are equal, but the impostor object is not the same as dr. seuss
# 'is' compares the identity of two objects
impostor == seuss == theodore, impostor is seuss

(True, False)

In [ ]:
# == vs is

# the most common use for 'is' is to check against a singleton like None
x = 1
x is None

False

In [8]:
# mutating items inside a tuple (previously discussed)

t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])

t1 == t2, id(t1[-1])

(True, 4428557248)

In [ ]:
t1[-1].append(100)

t1, id(t1[-1]), t1 == t2 # the identities of the objects in the tuple don't change, but the values can (if they're mutable)

((1, 2, [30, 40, 100, 100]), 4428557248, False)

In [ ]:
# copies are shallow by default

l1 = [3, [4, 5], (6, 7, 8)]
l2 = list(l1)
# the constructor (or l1[:]) creates a new list that is a copy of l1 and is equal but not the same
l2, l2 == l1, l2 is l1

# however, these are shallow copies. the references in both l1 and l2 point to the same objects

([3, [4, 5], (6, 7, 8)], True, False)

In [ ]:
l1.append(100) # this changes only l1 
l1[1].remove(4) # this changes both l1 and l2 since [4, 5] is being pointed to by both lists

l1, l2

([3, [5], (6, 7, 8), 100], [3, [5], (6, 7, 8)])

In [ ]:
l1[1] += [50] # += on a list changes the list in-place, which affects the shared list
l2[2] += (9, 10) # += on a tuple, however, creates a new tuple and rebinds l2[2] there, so l1 and l2 no longer have the same tuple

l1, l2

([3, [5, 50], (6, 7, 8), 100], [3, [5, 50], (6, 7, 8, 9, 10)])

In [17]:
# shallow vs deep copy
import copy

class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pickup(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

bus1 = Bus(['Alice', 'Bill', 'Coby', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

bus1.drop('Bill')
bus2.pickup('Emily')
id(bus1), id(bus2), id(bus3), id(bus1.passengers), id(bus2.passengers), id(bus3.passengers), bus1.passengers, bus2.passengers, bus3.passengers

# as you can see, all three busses are different objects.
# however, the passengers list is shared by bus1 and bus2 since bus2 is only a shallow copy of bus1.
# bus3 as a deep copy has its own passengers list that won't be affected by changes to bus1 or bus2

(4428770976,
 4428764592,
 4428774096,
 5039119424,
 5039119424,
 5039034560,
 ['Alice', 'Coby', 'David', 'Emily'],
 ['Alice', 'Coby', 'David', 'Emily'],
 ['Alice', 'Bill', 'Coby', 'David'])

In [ ]:
# cyclic references

a = [10, 20]
b = [a, 30]
a.append(b)
a, a[2][0][2][0][2][0][2][0] # lol

([10, 20, [[...], 30]], [10, 20, [[...], 30]])

In [27]:
c = copy.deepcopy(a)
c[2][0]

[10, 20, [[...], 30]]

In [ ]:
# functions can change mutable objects that they receive

def foo(a, b):
    a += b
    return a

x = 1
y = 2
foo(x, y), x, y # x is unchanged, ints are immutable

(3, 1, 2)

In [ ]:
a = [1, 2]
b = [3, 4]
foo(a, b), a, b # a is changed, lists are mutable

([1, 2, 3, 4], [1, 2, 3, 4], [3, 4])

In [ ]:
u = (10, 20)
v = (30, 40)

foo(u, v), u, v # u is unchanged, tuples are immutable

((10, 20, 30, 40), (10, 20), (30, 40))

In [32]:
# mutable types as defaults are not good
class HauntedBus:
    def __init__(self, passengers=[]): # here we alias passengers to the default list
        self.passengers = passengers # now self.passengers aliases to passengers. this will be bad later

    def pickup(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)
    

In [33]:
bus1 = HauntedBus(['Al', 'Bo'])

bus1.pickup('Cal')
bus1.drop('Al')

bus1.passengers

['Bo', 'Cal']

In [35]:
bus2 = HauntedBus() # invokes the default list
bus2.pickup('Boo') # this modifies the default list

bus3 = HauntedBus()
bus3.passengers, bus2.passengers is bus3.passengers

(['Boo', 'Boo'], True)

In [ ]:
# changing passed arguments

class TwilightBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers # this binds self.passengers to passengers, which will do weird things
            # instead, you should do:
            # self.passengers = list(passengers), which will make a copy

    def pickup(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

team = ['James','Reaves', 'Doncic']
spooky = TwilightBus(team)
spooky.drop('Reaves')

team # when dropping from the bus, it mutated the passed list as well. this violates the principle of least astonishment

['James', 'Doncic']

In [ ]:
# del

# del is a statement, not a function

# also, del deletes references, not objects. once an object has no references, the garbage collector will dispose of it
a = [1, 2]
b = a
del a # this deletes the reference a, not the object a is pointing to
b # [1, 2] still exists since b points to it. if we bind b to something else, then [1, 2] has no references and will be collected

[1, 2]

In [ ]:
# the above is accomplished with reference counters. once an object's reference count reaches zero, then it is destroyed

# we can use weakref.finalize to see this in action
from weakref import finalize

s1 = {1, 2 ,3}
s2 = s1

def bye(): # callback function called when an object is destroyed
    print("goodbye my friend...")

end = finalize(s1, bye) # this is a weak reference to s1, which does not increment the reference count
end.alive

True

In [47]:
del s1
end.alive

True

In [ ]:
s2 = 'bla'

goodbye my friend...


In [49]:
end.alive

False

## Chapter Summary

Objects have a value, identity, and type. Only the value can change over time (actually, you can change the type, but nah)

In [59]:
hey = [1, 2, 3]

def bla(stuff):
    new = stuff
    new[1] = 4

    return new

new = bla(hey)
new, hey, new is hey

([1, 4, 3], [1, 4, 3], True)